# Imports

In [23]:
import subprocess
import json
from pathlib import Path
from glob2 import glob
import os
from multiprocessing import Pool
from datetime import datetime
import pytz
from uuid import uuid4
import random

# Functions

In [24]:
import subprocess
import json
from pathlib import Path
from glob2 import glob
import os
from multiprocessing import Pool
from datetime import datetime
from uuid import uuid4
import random

In [30]:

def get_corpus_pieces(corpus):
    return sorted (os.path.basename(d).split(".")[0] for d in glob("inputs/slices/"+corpus+"/*.csv"))

def get_corpi():
    return [os.path.basename(d) for d in glob("inputs/slices/*")][1:]

def run_command(command):
    print(" ".join(command))    
    #res = subprocess.run(cmd, cwd="..")
    #res = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd="..")
    res = subprocess.run(command,stderr=subprocess.PIPE, cwd="..")

    if res.returncode != 0:
        print("Error in subprocess:\n")
        print(res.stderr)
        return;
            
def run_Experiment(commands,name,iterations, threads):
    uk_tz = pytz.timezone('Europe/London')
    id = datetime.now(uk_tz).strftime('%Y%m%d%H%M%S_')+ name
    #random.shuffle(commands)
    commands = [ ["stack","run","fullParse","--","-n",str(iterations), "-id", str(id)] + command for command in commands]
    with Pool(threads) as p:
        p.map(run_command, commands)

        
def stochastic_res_experiment(threads=28):
    commands = []
    corpi = get_corpi()
    beamWidthRange = range (1,40,2)
    reservoirSizeRange = range(100,1000,50)
    for corpus in corpi:
        pieces = get_corpus_pieces(corpus)
        for piece in pieces:
            for w in beamWidthRange:
                for r in reservoirSizeRange:
                    algo = "StochasticBeamSearch {} {}".format(w,r)
                    commands.append([ corpus, piece, algo])
    
    print("Running {} Stochastic Beam Search experiments, with beamwidth {} and reservoirSize {}".format(len(commands), beamWidthRange, reservoirSizeRange))

    run_Experiment(commands, "stochastic-all-params", threads)

def dual_stochastic_res_experiment(inputs, beamWidthRange, reservoirSizeRange, iterations=1, threads=28):
    commands = []
    corpi = get_corpi()

    for (corpus, piece) in inputs:
        for w in beamWidthRange:
            for r in reservoirSizeRange:
                algo = "DualStochasticBeamSearch {} {}".format(w,r)
                commands.append([ corpus, piece, algo])
    
    print("Running {} Stochastic Beam Search experiments, with beamwidth {} and reservoirSize {}".format(len(commands), beamWidthRange, reservoirSizeRange))
    random.shuffle(commands)
    run_Experiment(commands, "dual-stochastic-res-params", iterations,threads)

def all_beam_searches_experiment(inputs, threads=28):
    beamWidths = range (6,36,3)
    for (corpus, piece) in inputs:
        for beamWidth in beamWidths:
            algo = "DualStochasticBeamSearch {} {}".format(int(beamWidth/3),reservoirSize)
            algo2 = "DualStochasticBeamSearch {} {}".format(int(beamWidth/3),reservoirSize)
            commands.append([ corpus, piece, algo])
            commands.append([ corpus, piece, algo2])
            

    
def stochastic_res_lim_experiment(threads=28):
    commands = []
    corpi = get_corpi()
    beamWidthRange = range (7,14,1)
    limitRange = range(1,20,1)
    reservoirSize = 700
    for corpus in corpi:
        pieces = get_corpus_pieces(corpus)
        for piece in pieces:
            for w in beamWidthRange:
                for l in limitRange:
                    algo = "StochasticBeamSearchLimited {} {} {}".format(w,reservoirSize,l)
                    commands.append([ corpus, piece, algo])
    
    print("Running {} Stochastic Beam Search Limited experiments, with beamwidth {}, reservoirSize {} and spread note limit {}".format(len(commands), beamWidthRange, reservoirSize, limitRange))
    print("Experiment will take approx {}-{} hours".format(commands/ 28 * 0.1),commands/ 28 * 0.27)

    run_Experiment(commands, "stochastic-limited-fixed-res", threads)

In [31]:
get_corpi()

['tests', 'grieg_lyric_pieces', 'schumann_kinderszenen', 'chopin_mazurkas']

# Run Experiments

In [32]:
## with 360 runs at 28 it took like 10 seconds 24 seconds at 56 threads, and 15 wihth 28.

In [33]:
if __name__ == '__main__':
    beamWidthRange = range (5,40,5)
    reservoirSizeRange = range(10,300,20)
    dual_stochastic_res_experiment([("grieg_lyric_pieces", "op12n08")],beamWidthRange,reservoirSizeRange, iterations=1,threads=28)

Running 105 Stochastic Beam Search experiments, with beamwidth range(5, 40, 5) and reservoirSize range(10, 300, 20)
stack run fullParse -- -n 1 -id 20230417230541_dual-stochastic-res-params grieg_lyric_pieces op12n01 DualStochasticBeamSearch 5 110stack run fullParse -- -n 1 -id 20230417230541_dual-stochastic-res-params grieg_lyric_pieces op12n01 DualStochasticBeamSearch 15 150stack run fullParse -- -n 1 -id 20230417230541_dual-stochastic-res-params grieg_lyric_pieces op12n01 DualStochasticBeamSearch 15 110stack run fullParse -- -n 1 -id 20230417230541_dual-stochastic-res-params grieg_lyric_pieces op12n01 DualStochasticBeamSearch 20 250stack run fullParse -- -n 1 -id 20230417230541_dual-stochastic-res-params grieg_lyric_pieces op12n01 DualStochasticBeamSearch 10 230stack run fullParse -- -n 1 -id 20230417230541_dual-stochastic-res-params grieg_lyric_pieces op12n01 DualStochasticBeamSearch 15 30stack run fullParse -- -n 1 -id 20230417230541_dual-stochastic-res-params grieg_lyric_pieces o

In [7]:
#if __name__ == '__main__':
    #algo_experiment()
    #beamwidth_experiment()